In [91]:
import pandas as pd

### Load data sets for airbnb, rental, and home

In [139]:
airbnb = pd.read_csv("../data/clean/inside_airbnb_clean.gz")
rental = pd.read_csv("../data/clean/nyc_rental_prices.csv")
home = pd.read_csv("../data/clean/zhome_value.csv")
rental_home_avg  = pd.read_csv("./nyc_airbnb_home_rental_data.csv")

In [127]:
airbnb.head()

,id,last_scraped,host_id,host_name,neighbourhood,latitude,longitude,room_type,bedrooms,availability_365,calculated_host_listings_count,geom,zipcode,borough,year,availability_pct
0,2539,2022-09-07,2787,John,Kensington,40.64529,-73.97238,Private room,1.0,356,9,"40.64529,-73.97238",11218,BROOKLYN,2022,97.534247
1,5121,2022-09-07,7356,Garon,Bedford-Stuyvesant,40.68535,-73.95512,Private room,1.0,335,2,"40.68535,-73.95512",11238,BROOKLYN,2022,91.780822
2,45910,2022-09-07,204539,Mark,Ridgewood,40.70309,-73.89963,Entire home/apt,5.0,365,6,"40.70309,-73.89963",11385,QUEENS,2022,100.000000
3,5136,2022-09-07,7378,Rebecca,Sunset Park,40.66265,-73.99454,Entire home/apt,2.0,179,1,"40.66265,-73.99454",11215,BROOKLYN,2022,49.041096
4,45935,2022-09-07,204586,L,Mott Haven,40.80635,-73.92201,Private room,1.0,83,1,"40.80635,-73.92201",10454,BRONX,2022,22.739726


In [128]:
rental.head()

,rent,borough,zipcode,year
0,16000.0,MANHATTAN,10013,2022
1,8250.0,MANHATTAN,10012,2022
2,2200.0,MANHATTAN,10031,2022
3,13000.0,MANHATTAN,10011,2022
4,55000.0,MANHATTAN,10065,2022


In [129]:
home.head()

,zipcode,state,city,borough,year,home_value
0,11368,NY,New York,QUEENS,2002,249885.0
1,11385,NY,New York,QUEENS,2002,333404.0
2,11236,NY,New York,BROOKLYN,2002,255687.0
3,11208,NY,New York,BROOKLYN,2002,235676.0
4,10467,NY,New York,BRONX,2002,174696.0


In [130]:
rental_home_avg.head()

,zipcode,year,borough,rent,home_value,count
0,10001,2015,MANHATTAN,4010.000000,2.280604e+06,350
1,10001,2016,MANHATTAN,4022.983333,2.369453e+06,666
2,10001,2017,MANHATTAN,3958.383333,2.271804e+06,418
3,10001,2018,MANHATTAN,4001.066667,2.351997e+06,491
4,10001,2019,MANHATTAN,4145.266667,2.380720e+06,275


### Prep Rental/Home Data for avg merge to count df

In [140]:
rental_home_avg = rental_home_avg.drop(columns=["count"]).rename(columns={"rent": "rent_avg", "home_value": "house_avg"})

In [141]:
rental_home_avg.head()

,zipcode,year,borough,rent_avg,house_avg
0,10001,2015,MANHATTAN,4010.000000,2.280604e+06
1,10001,2016,MANHATTAN,4022.983333,2.369453e+06
2,10001,2017,MANHATTAN,3958.383333,2.271804e+06
3,10001,2018,MANHATTAN,4001.066667,2.351997e+06
4,10001,2019,MANHATTAN,4145.266667,2.380720e+06
...,...,...,...,...,...
729,11435,2022,QUEENS,1923.566667,6.558268e+05
730,11436,2022,QUEENS,2100.000000,6.106205e+05
731,11691,2022,QUEENS,2706.150000,6.434812e+05
732,11692,2022,QUEENS,2700.000000,5.621005e+05


### Get yearly count for each dataset
Function that groups by `groupby_columns` and uses `count_column` to count occurances with
 `count_column` column being renamed to `renamed_column`. Ends with output columns being printed

In [96]:
def groupby_count(df, groupby_columns, count_column, renamed_column):
    count_data = df.groupby(groupby_columns)[count_column].count().to_frame().reset_index().rename(columns={count_column: renamed_column})
    print("{} Columns:".format(renamed_column), count_data.columns)
    print("count", len(count_data))
    return count_data

In [97]:
groupby_columns = ["year", "borough", "zipcode"]

In [98]:
airbnb_count = groupby_count(airbnb, groupby_columns, "id", "airbnb_count")
rental_count = groupby_count(rental, groupby_columns, "rent", "rental_count")
home_count = groupby_count(home, groupby_columns, "home_value", "home_count")

airbnb_count Columns: Index(['year', 'borough', 'zipcode', 'airbnb_count'], dtype='object')
count 1429
rental_count Columns: Index(['year', 'borough', 'zipcode', 'rental_count'], dtype='object')
count 854
home_count Columns: Index(['year', 'borough', 'zipcode', 'home_count'], dtype='object')
count 3633


In [146]:
merged_df = pd.merge(airbnb_count, rental_count, on=groupby_columns)
merged_df = pd.merge(merged_df, home_count, on=groupby_columns)
merged_df = pd.merge(merged_df, rental_home_avg, on=groupby_columns)
merged_df

,year,borough,zipcode,airbnb_count,rental_count,home_count,rent_avg,house_avg
0,2015,BRONX,10457,51,10,12,1301.130000,324363.916667
1,2015,BRONX,10458,41,10,12,1412.190000,284365.833333
2,2015,BRONX,10461,49,10,12,1320.460000,409960.166667
3,2015,BRONX,10462,42,10,12,1372.010000,124176.000000
4,2015,BRONX,10463,48,10,12,2072.790000,333158.666667
...,...,...,...,...,...,...,...,...
729,2022,STATEN_ISLAND,10306,35,11,10,1997.690909,670325.100000
730,2022,STATEN_ISLAND,10308,18,2,10,2432.900000,650727.100000
731,2022,STATEN_ISLAND,10309,6,2,10,1925.650000,713093.500000
732,2022,STATEN_ISLAND,10312,11,12,10,1999.758333,673293.500000


In [147]:
print("airbnb_count: {}".format(merged_df["airbnb_count"].sum()), "rental_count: {}".format(merged_df["rental_count"].sum()), "home_count: {}".format(merged_df["home_count"].sum()), sep="\n")

airbnb_count: 296464
rental_count: 8153
home_count: 8500


In [149]:
print("min year: {}".format(merged_df["year"].min()), "max_year: {}".format(merged_df["year"].max()), sep="\n")

min year: 2015
max_year: 2022


In [151]:
merged_df.isna().any()

year            False
borough         False
zipcode         False
airbnb_count    False
rental_count    False
home_count      False
rent_avg        False
house_avg       False
dtype: bool

### Export CSV

In [152]:
# merged_df.to_csv("airbnb_rental_home_data.csv", index=False)
# print("Created csv")

Created csv
